<a href="https://colab.research.google.com/github/Mayur619/avc-detection/blob/notebooks/Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pathlib

import matplotlib
import matplotlib.pyplot as plt

import io
import scipy.misc
import numpy as np
import csv
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from six.moves.urllib.request import urlopen

import tensorflow as tf
import tensorflow_hub as hub

tf.get_logger().setLevel('ERROR')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# @title Run this!!

def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: the file path to the image

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  image = None
  if(path.startswith('http')):
    response = urlopen(path)
    image_data = response.read()
    image_data = BytesIO(image_data)
    image = Image.open(image_data)
  else:
    image_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(BytesIO(image_data))

  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (1, im_height, im_width, 3)).astype(np.uint8)


ALL_MODELS = {
'CenterNet HourGlass104 512x512' : 'https://tfhub.dev/tensorflow/centernet/hourglass_512x512/1',
'CenterNet HourGlass104 Keypoints 512x512' : 'https://tfhub.dev/tensorflow/centernet/hourglass_512x512_kpts/1',
'CenterNet HourGlass104 1024x1024' : 'https://tfhub.dev/tensorflow/centernet/hourglass_1024x1024/1',
'CenterNet HourGlass104 Keypoints 1024x1024' : 'https://tfhub.dev/tensorflow/centernet/hourglass_1024x1024_kpts/1',
'CenterNet Resnet50 V1 FPN 512x512' : 'https://tfhub.dev/tensorflow/centernet/resnet50v1_fpn_512x512/1',
'CenterNet Resnet50 V1 FPN Keypoints 512x512' : 'https://tfhub.dev/tensorflow/centernet/resnet50v1_fpn_512x512_kpts/1',
'CenterNet Resnet101 V1 FPN 512x512' : 'https://tfhub.dev/tensorflow/centernet/resnet101v1_fpn_512x512/1',
'CenterNet Resnet50 V2 512x512' : 'https://tfhub.dev/tensorflow/centernet/resnet50v2_512x512/1',
'CenterNet Resnet50 V2 Keypoints 512x512' : 'https://tfhub.dev/tensorflow/centernet/resnet50v2_512x512_kpts/1',
'EfficientDet D0 512x512' : 'https://tfhub.dev/tensorflow/efficientdet/d0/1',
'EfficientDet D1 640x640' : 'https://tfhub.dev/tensorflow/efficientdet/d1/1',
'EfficientDet D2 768x768' : 'https://tfhub.dev/tensorflow/efficientdet/d2/1',
'EfficientDet D3 896x896' : 'https://tfhub.dev/tensorflow/efficientdet/d3/1',
'EfficientDet D4 1024x1024' : 'https://tfhub.dev/tensorflow/efficientdet/d4/1',
'EfficientDet D5 1280x1280' : 'https://tfhub.dev/tensorflow/efficientdet/d5/1',
'EfficientDet D6 1280x1280' : 'https://tfhub.dev/tensorflow/efficientdet/d6/1',
'EfficientDet D7 1536x1536' : 'https://tfhub.dev/tensorflow/efficientdet/d7/1',
'SSD MobileNet v2 320x320' : 'https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2',
'SSD MobileNet V1 FPN 640x640' : 'https://tfhub.dev/tensorflow/ssd_mobilenet_v1/fpn_640x640/1',
'SSD MobileNet V2 FPNLite 320x320' : 'https://tfhub.dev/tensorflow/ssd_mobilenet_v2/fpnlite_320x320/1',
'SSD MobileNet V2 FPNLite 640x640' : 'https://tfhub.dev/tensorflow/ssd_mobilenet_v2/fpnlite_640x640/1',
'SSD ResNet50 V1 FPN 640x640 (RetinaNet50)' : 'https://tfhub.dev/tensorflow/retinanet/resnet50_v1_fpn_640x640/1',
'SSD ResNet50 V1 FPN 1024x1024 (RetinaNet50)' : 'https://tfhub.dev/tensorflow/retinanet/resnet50_v1_fpn_1024x1024/1',
'SSD ResNet101 V1 FPN 640x640 (RetinaNet101)' : 'https://tfhub.dev/tensorflow/retinanet/resnet101_v1_fpn_640x640/1',
'SSD ResNet101 V1 FPN 1024x1024 (RetinaNet101)' : 'https://tfhub.dev/tensorflow/retinanet/resnet101_v1_fpn_1024x1024/1',
'SSD ResNet152 V1 FPN 640x640 (RetinaNet152)' : 'https://tfhub.dev/tensorflow/retinanet/resnet152_v1_fpn_640x640/1',
'SSD ResNet152 V1 FPN 1024x1024 (RetinaNet152)' : 'https://tfhub.dev/tensorflow/retinanet/resnet152_v1_fpn_1024x1024/1',
'Faster R-CNN ResNet50 V1 640x640' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet50_v1_640x640/1',
'Faster R-CNN ResNet50 V1 1024x1024' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet50_v1_1024x1024/1',
'Faster R-CNN ResNet50 V1 800x1333' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet50_v1_800x1333/1',
'Faster R-CNN ResNet101 V1 640x640' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet101_v1_640x640/1',
'Faster R-CNN ResNet101 V1 1024x1024' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet101_v1_1024x1024/1',
'Faster R-CNN ResNet101 V1 800x1333' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet101_v1_800x1333/1',
'Faster R-CNN ResNet152 V1 640x640' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet152_v1_640x640/1',
'Faster R-CNN ResNet152 V1 1024x1024' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet152_v1_1024x1024/1',
'Faster R-CNN ResNet152 V1 800x1333' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet152_v1_800x1333/1',
'Faster R-CNN Inception ResNet V2 640x640' : 'https://tfhub.dev/tensorflow/faster_rcnn/inception_resnet_v2_640x640/1',
'Faster R-CNN Inception ResNet V2 1024x1024' : 'https://tfhub.dev/tensorflow/faster_rcnn/inception_resnet_v2_1024x1024/1',
'Mask R-CNN Inception ResNet V2 1024x1024' : 'https://tfhub.dev/tensorflow/mask_rcnn/inception_resnet_v2_1024x1024/1'
}



## Visualization tools

To visualize the images with the proper detected boxes, keypoints and segmentation, we will use the TensorFlow Object Detection API. To install it we will clone the repo.

In [ ]:
# Clone the tensorflow models repository
!git clone --depth 1 https://github.com/tensorflow/models

Intalling the Object Detection API

In [ ]:
%%bash
sudo apt install -y protobuf-compiler
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .


Now we can import the dependencies we will need later

In [ ]:
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import ops as utils_ops

%matplotlib inline

### Load label map data (for plotting).

Label maps correspond index numbers to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine.

We are going, for simplicity, to load from the repository that we loaded the Object Detection API code

In [ ]:
PATH_TO_LABELS = './models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

## Build a detection model and load pre-trained model weights

Here we will choose witch Object Detection model we will use.
Select the architecture and it will be loaded automatically.
If you want to change the model to try other architectures later, just change the next cell and execute following ones.

**Tip:** if you want to read more details about the selected model, you can follow the link (model handle) and read aditional documentation on TF Hub. After you select a model, we will print the handle to make it easier.

In [ ]:
#@title Model Selection { display-mode: "form", run: "auto" }
model_display_name = 'SSD MobileNet V1 FPN 640x640' # @param ['CenterNet HourGlass104 512x512','CenterNet HourGlass104 Keypoints 512x512','CenterNet HourGlass104 1024x1024','CenterNet HourGlass104 Keypoints 1024x1024','CenterNet Resnet50 V1 FPN 512x512','CenterNet Resnet50 V1 FPN Keypoints 512x512','CenterNet Resnet101 V1 FPN 512x512','CenterNet Resnet50 V2 512x512','CenterNet Resnet50 V2 Keypoints 512x512','EfficientDet D0 512x512','EfficientDet D1 640x640','EfficientDet D2 768x768','EfficientDet D3 896x896','EfficientDet D4 1024x1024','EfficientDet D5 1280x1280','EfficientDet D6 1280x1280','EfficientDet D7 1536x1536','SSD MobileNet v2 320x320','SSD MobileNet V1 FPN 640x640','SSD MobileNet V2 FPNLite 320x320','SSD MobileNet V2 FPNLite 640x640','SSD ResNet50 V1 FPN 640x640 (RetinaNet50)','SSD ResNet50 V1 FPN 1024x1024 (RetinaNet50)','SSD ResNet101 V1 FPN 640x640 (RetinaNet101)','SSD ResNet101 V1 FPN 1024x1024 (RetinaNet101)','SSD ResNet152 V1 FPN 640x640 (RetinaNet152)','SSD ResNet152 V1 FPN 1024x1024 (RetinaNet152)','Faster R-CNN ResNet50 V1 640x640','Faster R-CNN ResNet50 V1 1024x1024','Faster R-CNN ResNet50 V1 800x1333','Faster R-CNN ResNet101 V1 640x640','Faster R-CNN ResNet101 V1 1024x1024','Faster R-CNN ResNet101 V1 800x1333','Faster R-CNN ResNet152 V1 640x640','Faster R-CNN ResNet152 V1 1024x1024','Faster R-CNN ResNet152 V1 800x1333','Faster R-CNN Inception ResNet V2 640x640','Faster R-CNN Inception ResNet V2 1024x1024','Mask R-CNN Inception ResNet V2 1024x1024']
model_handle = ALL_MODELS[model_display_name]

print('Selected model:'+ model_display_name)
print('Model Handle at TensorFlow Hub: {}'.format(model_handle))

## Loading the selected model from TensorFlow Hub

Here we just need the model handle that was selected and use the Tensorflow Hub library to load it to memory.


In [ ]:
print('loading model...')
hub_model = hub.load(model_handle)
print('model loaded!')

## Loading an image

Let's try the model on a simple image. To help with this, we provide a list of test images.

Here are some simple things to try out if you are curious:
* Try running inference on your own images, just upload them to colab and load the same way it's done in the cell below.
* Modify some of the input images and see if detection still works.  Some simple things to try out here include flipping the image horizontally, or converting to grayscale (note that we still expect the input image to have 3 channels).

**Be careful:** when using images with an alpha channel, the model expect 3 channels images and the alpha will count as a 4th.



In [ ]:
# Animal label ids
animal_label_ids = {"cow":21,"dog":18,"bear":23}

In [ ]:
for animal in animal_label_ids.keys():
  print(animal)
  CURRENT_ANIMAL_DIR = animal
  BASE_PATH = "/content/drive/My Drive/AnimalVehicleCollision/data/images/"+CURRENT_ANIMAL_DIR
  results_file = open(os.path.join(BASE_PATH,"bounding_box.csv"),"w")
  results_file.write("image_name,x,y,width,height\n")
  image_files = [file for file in os.listdir(BASE_PATH) if file.endswith('.jpg') or file.endswith('.jpeg') or file.endswith('.png') or file.endswith('.webp')]
  print("Total {0} images to be processed".format(len(image_files)))
  for i,image_name in enumerate(image_files):
    try:
      image_np = load_image_into_numpy_array(BASE_PATH+"/"+image_name)
      results = hub_model(image_np)
      index=np.where(results['detection_classes'][0]==animal_label_ids[CURRENT_ANIMAL_DIR])[0][0]
      data_to_write = image_name + ','+','.join([str(s) for s in results['detection_boxes'][0][index].numpy()])
      results_file.write(data_to_write+"\n")
      print("\r",end="")
      print("\rProcessed {0} images".format(i),end="")
    except (ValueError,IndexError) as e:
      continue
  results_file.close()

In [ ]:
with open(os.path.join(BASE_PATH,'bounding_box.csv'),'r') as csv_file:
  valid_files = set() 
  for row in csv.reader(csv_file,delimiter=','):
    valid_files.add(row[0])
  for image_file in image_files:
    if image_file not in valid_files:
      print(image_file)
      os.remove(os.path.join(BASE_PATH,image_file))
      print('Removing {0}'.format(image_file))

In [ ]:
import shutil

for image in os.listdir(BASE_PATH):
  if image[0]=='m':
    shutil.copyfile(os.path.join(BASE_PATH,image),"/content/drive/My Drive/AnimalVehicleCollision/bear_1/"+image)
  else:
    shutil.copyfile(os.path.join(BASE_PATH,image),"/content/drive/My Drive/AnimalVehicleCollision/bear_2/"+image)

In [ ]:
len(os.listdir("/content/drive/My Drive/AnimalVehicleCollision/bear_2/"))

In [ ]:
results["detection_boxes"][0][0].numpy()

## Doing the inference

To do the inference we just need to call our TF Hub loaded model.

Things you can try:
* Print out `result['detection_boxes']` and try to match the box locations to the boxes in the image.  Notice that coordinates are given in normalized form (i.e., in the interval [0, 1]).
* inspect other output keys present in the result. A full documentation can be seen on the models documentation page (pointing your browser to the model handle printed earlier)

## Train test split

In [ ]:
import pandas as pd
import os
animal_categories=["cow","dog","bear"]
TRAIN_SIZE = 0.8
VALIDATION_SIZE = 0.1
TEST_SIZE = 0.1
BASE_PATH_BB = "/content/drive/My Drive/AnimalVehicleCollision/data/bounding_boxes_and_path_coordinates"

In [ ]:
bounding_boxes=[]
path_coordinates=[]
for i in range(len(animal_categories)):
  bounding_box_df = pd.read_csv(os.path.join(BASE_PATH_BB,animal_categories[i],"bounding_box.csv"))
  bounding_box_df["label"] = [i]*bounding_box_df.shape[0]
  bounding_boxes.append(bounding_box_df)
  path_coordinates.append(pd.read_csv(os.path.join(BASE_PATH_BB,animal_categories[i],"path_coordinates.csv")))
bounding_box_df = pd.concat(bounding_boxes)
path_coordinates_df = pd.concat(path_coordinates)
merged_df = pd.merge(left = bounding_box_df, right = path_coordinates_df, left_on = "image_name", right_on = "image_name")
merged_df['normalized_x'].fillna(merged_df['normalized_x'].mean(),inplace = True)
merged_df['normalized_y'].fillna(merged_df['normalized_y'].mean(),inplace = True)
merged_df=merged_df.rename(columns = {"x":"box_x","y":"box_y","normalized_x":"path_x","normalized_y":"path_y"})
merged_df=merged_df.sample(frac=1).reset_index(drop=True)

In [ ]:
train_data = merged_df.iloc[:round(TRAIN_SIZE*merged_df.shape[0]),:]
validation_data = merged_df.iloc[round(TRAIN_SIZE*merged_df.shape[0]):round(TRAIN_SIZE*merged_df.shape[0])+round(VALIDATION_SIZE*merged_df.shape[0]),:]
test_data = merged_df.iloc[round(TRAIN_SIZE*merged_df.shape[0])+round(VALIDATION_SIZE*merged_df.shape[0]):,:]

train_data.to_csv("/content/drive/My Drive/AnimalVehicleCollision/data/train.csv")
validation_data.to_csv("/content/drive/My Drive/AnimalVehicleCollision/data/validation.csv")
test_data.to_csv("/content/drive/My Drive/AnimalVehicleCollision/data/test.csv")

In [ ]:
import pandas as pd

training = pd.read_csv("/content/drive/My Drive/AnimalVehicleCollision/data/train.csv")
validation = pd.read_csv("/content/drive/My Drive/AnimalVehicleCollision/data/validation.csv")
test = pd.read_csv("/content/drive/My Drive/AnimalVehicleCollision/data/test.csv")

In [ ]:
training.head()

In [ ]:
change_orientation={3:1,1:2,2:3}
training["orientation"]=training["orientation"].replace(change_orientation)
validation["orientation"]=validation["orientation"].replace(change_orientation)
test["orientation"]=test["orientation"].replace(change_orientation)

In [ ]:
training=training.loc[:,"image_name":]
validation=validation.loc[:,"image_name":]
test=test.loc[:,"image_name":]

In [ ]:
training.to_csv("/content/drive/My Drive/AnimalVehicleCollision/data/train.csv",index=False)
validation.to_csv("/content/drive/My Drive/AnimalVehicleCollision/data/validation.csv",index=False)
test.to_csv("/content/drive/My Drive/AnimalVehicleCollision/data/test.csv",index=False)